In [1]:
from bs4 import BeautifulSoup

In [60]:
import requests

In [63]:
from tqdm import tqdm_notebook

### link_list twitch metric

In [3]:
df_streamList = pd.DataFrame(columns = ["Name", "Link"])

In [4]:
for page in [1, 2]:
    rank_url = "https://www.twitchmetrics.net/channels/viewership?lang=ko&page=" + str(page)
    response = requests.get(rank_url)
    dom = BeautifulSoup(response.content, "html.parser")
    dom_stream_list = dom.select(".list-group .list-group-item .d-flex.mb-2.flex-wrap a")
    
    for dom_stream in dom_stream_list:
        link = dom_stream.get("href")
        name = dom_stream.text.strip().split(" ")[0]
        df_streamList.loc[len(df_streamList)] = {"Name" : name, "Link" : link}

In [5]:
df_streamList.head()

,Name,Link
0,풍월량,/c/103825127-hanryang1125
1,따효니,/c/102381201-ddahyoni
2,overwatchleague_kr,/c/188864445-overwatchleague_kr
3,서새봄냥,/c/30904062-saddummy
4,LCK_Korea,/c/190835892-lck_korea


### Function

In [56]:
li_stream_basic_info = ["Name", "followers", "followers_diff", \
                       "viwer_avg", "viwer_avg_diff", \
                       "peak_viwer", "peak_viwer_diff", \
                       "hour", "hour_diff", \
                       "mature_content", "language", "join_date"]

### get twitch metric info

##### Basic info

In [57]:
from selenium import webdriver

In [64]:
def crawling_stream_basic_info(df_list):
    df_stream_basic_info = pd.DataFrame(columns = li_stream_basic_info)
    driver = webdriver.Chrome()
    
    for idx, Link in tqdm_notebook(enumerate(df_list["Link"])):
        s_stream_basic_info = pd.Series(index = li_stream_basic_info)
        
        stream_url = "https://www.twitchmetrics.net" + Link
        driver.get(stream_url)

        stream_details = driver.find_elements_by_css_selector("div.card-block dd.col-6")
        stream_this_week = driver.find_elements_by_css_selector("div.card-block samp")
        
        s_stream_basic_info["Name"] = df_list["Name"][idx]
        
        s_stream_basic_info["followers"] = stream_this_week[0].text
        s_stream_basic_info["followers_diff"] = stream_this_week[1].text.split(" ")[0]
        s_stream_basic_info["viwer_avg"] = stream_this_week[2].text
        s_stream_basic_info["viwer_avg_diff"] = stream_this_week[3].text.split(" ")[0]
        s_stream_basic_info["peak_viwer"] = stream_this_week[4].text
        s_stream_basic_info["peak_viwer_diff"] = stream_this_week[5].text.split(" ")[0]
        s_stream_basic_info["hour"] = stream_this_week[6].text
        s_stream_basic_info["hour_diff"] = stream_this_week[7].text.split(" ")[0]

        s_stream_basic_info["mature_content"] = stream_details[0].text
        s_stream_basic_info["language"] = stream_details[1].text
        s_stream_basic_info["join_date"] = stream_details[2].text
        
        df_stream_basic_info.loc[len(df_stream_basic_info)] = s_stream_basic_info
        
    driver.close()
    
    return df_stream_basic_info

In [66]:
df_stream_basic_info = crawling_stream_basic_info(df_streamList)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [71]:
df_stream_basic_info.to_csv("data/stream_basic_info.csv",index = False)

##### crawling_viewship_value

In [102]:
li_stream_viewship = ["time", "viewer", "theme"]

In [103]:
def crawling_stream_viewship(idx, Name, Link):
    stream_viewship_url = "https://www.twitchmetrics.net" + Link + "/recent_viewership_values"
    df_stream_viewship = pd.DataFrame(columns = li_stream_viewship)
    
    dom_stream_viewship = requests.get(stream_viewship_url)
    stream_viewships = dom_stream_viewship.json()
    
    for viewship in stream_viewships:
        df_stream_viewship.loc[len(df_stream_viewship)] = {"time" : viewship[0], "viewer" : viewship[1], "theme" : viewship[2]}
    
    path_save = "data/viewship/" + str(idx) + "_" + Name + '.csv'
    
    df_stream_viewship.to_csv(path_save, index = False)

##### crawling_history

In [104]:
li_stream_history = ["start_time", "end_time", "theme", "elapsed_time", "viewer_avg", "total_follower", "total_viewer"]

In [105]:
def crawling_stream_history(idx, Name, Link):
    stream_history_url = "https://www.twitchmetrics.net" + Link + "/stream_history_values"
    df_stream_history = pd.DataFrame(columns = li_stream_history)
    
    dom_stream_history = requests.get(stream_history_url)
    stream_histories = dom_stream_history.json()
    
    for history in stream_histories:
        df_stream_history.loc[len(df_stream_history)] = {"start_time" : history[0], "end_time" : history[1], \
                                                         "theme" : history[2], "elapsed_time" : history[3], \
                                                         "viewer_avg" : history[4], "total_follower" : history[5], \
                                                         "total_viewer" : history[6]}
    
    path_save = "data/history/" + str(idx) + "_" + Name + '.csv'
    
    df_stream_history.to_csv(path_save, index = False)

### Crawling

In [106]:
for idx in tqdm_notebook(range(len(df_streamList))):
    name = df_streamList["Name"][idx]
    link = df_streamList["Link"][idx]
    
    crawling_stream_viewship(idx, name, link)
    crawling_stream_history(idx, name, link)

HBox(children=(IntProgress(value=0), HTML(value='')))

- 방송 시작 시간
- 방송 종료 시간
- 방송 종류
- 방송 시간(second)
- viewer(average)
- total followers
- total viewer